In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Define data generators with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_dir = '/Users/isabel/desktop/CHEST-XRAY/chest_xray/train'
val_dir = '/Users/isabel/desktop/CHEST-XRAY/chest_xray/test'

# Batch size (increase for better convergence)
batch_size = 64

try:
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(224, 224),  # VGG16 input size
        batch_size=batch_size,
        class_mode='binary' 
    )

    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary'  
    )
except FileNotFoundError as e:
    print("Error: Directory not found. Please check your file paths.")
    raise e

# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Fine-tune the last few layers of the base model
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Create a new model on top of the pre-trained base with regularisation
model = Sequential([
    base_model,
    Flatten(),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# Define a custom learning rate scheduler function
def lr_scheduler(epoch):
    initial_learning_rate = 1e-4
    lr = initial_learning_rate * tf.math.pow(0.9, epoch // 5)  # Decay every 5 epochs
    return lr

# Compile the model with appropriate loss + metrics
model.compile(optimizer=Adam(learning_rate=lr_scheduler(0)), loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping and reduce learning rate on plateau
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6)

# Train the model
epochs = 15  
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[early_stopping, reduce_lr_on_plateau]
)


Found 5232 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Epoch 1/15
81/81 [==============================] - 103s 1s/step - loss: 0.4309 - accuracy: 0.8440 - val_loss: 0.2569 - val_accuracy: 0.9028 - lr: 1.0000e-04
Epoch 2/15
81/81 [==============================] - 99s 1s/step - loss: 0.2303 - accuracy: 0.9292 - val_loss: 0.3423 - val_accuracy: 0.8993 - lr: 1.0000e-04
Epoch 3/15
81/81 [==============================] - 100s 1s/step - loss: 0.1884 - accuracy: 0.9421 - val_loss: 0.5102 - val_accuracy: 0.8455 - lr: 1.0000e-04
Epoch 4/15
81/81 [==============================] - 100s 1s/step - loss: 0.1560 - accuracy: 0.9516 - val_loss: 0.5244 - val_accuracy: 0.8472 - lr: 1.0000e-04
Epoch 5/15
81/81 [==============================] - 100s 1s/step - loss: 0.1527 - accuracy: 0.9539 - val_loss: 0.3870 - val_accuracy: 0.8993 - lr: 1.0000e-04
Epoch 6/15
81/81 [==============================] - 100s 1s/step - loss: 0.1406 - accuracy: 0.9634 - val_loss: 0.3326 - val_accuracy: 0.9115 - lr: 1.0000e-04
Epoch 7/15
81/81 [==============================] - 1

In [3]:
# Evaluate the model on the validation data
eval_result = model.evaluate(val_generator)

# Print the evaluation result
print("Validation Loss:", eval_result[0])
print("Validation Accuracy:", eval_result[1])

10/10 [==============================] - 9s 873ms/step - loss: 0.2254 - accuracy: 0.9215
Validation Loss: 0.2253951132297516
Validation Accuracy: 0.9214743971824646


In [6]:
# After training is complete
model.save("CHESTXRAY.KERAS")

INFO:tensorflow:Assets written to: CHESTXRAY.KERAS/assets


INFO:tensorflow:Assets written to: CHESTXRAY.KERAS/assets
